# Real Madrid Maç Tahmin Projesi



Bu proje, Real Madrid futbol takımının maç sonuçlarını tahmin etmek ve maç istatistiklerini görselleştirmek amacıyla oluşturulmuştur.

## Veri Toplama ve İşleme

### Web Scraping ve Veri Ön İşleme
Bu bölümde, [Sky Sports](https://www.skysports.com/real-madrid-results/2023-24) web sitesinden Real Madrid maç sonuçlarını çekme ve işleme işlemleri yapılmıştır.

- `scrape_and_preprocess_matches()` fonksiyonu, belirli bir tarih aralığındaki maç verilerini çeker ve işler.
- Çekilen veriler, 'processed_matches.csv' dosyasına kaydedilir.

## Model Eğitimi ve Değerlendirmesi

Bu kısımda, çekilen maç verileri üzerinden farklı makine öğrenimi modelleri eğitilmiş ve performansları değerlendirilmiştir.

- Eğitilen modeller arasında Random Forest, Gradient Boosting gibi çeşitli regrasiyon veya sınıflandırma algoritmaları bulunmaktadır.
- model, 'model.pkl' dosyasına kaydedilmiştir.

## Streamlit Uygulaması

Bu bölümde, kullanıcı arayüzü oluşturulmuş Real Madrid maç tahminleri yapılmıştır.

- Kullanıcı, bir takım adını seçerek Real Madrid'ın o takıma karşı kazanma kayıbetme veya berabere kalma olasılıklarını görebilir.
- Tahmin sonuçları ve maç istatistikleri görsel olarak sunulmuştur.

[matchresultguess](https://matchresultguess.streamlit.app/) sitesinde modelin gerçek dünyada çalışmasını takip edebilriz


In [77]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def scrape_and_preprocess_matches():
    # Maç verilerinin toplanacağı boş bir liste oluşturulur.
    all_match_data = []

    # Şu anki yıl alınır
    current_year = datetime.now().year

    # Yılları tersten (en son yıldan ilk yıla doğru) dolaşır.
    for year in range(current_year, 1999, -1):  
        # Yıl aralığını string formatında oluşturur (örneğin, '2023-2024')
        year_str = f'{year}-{year+1:02d}'
        
        # Skysports sitesinden Real Madrid'in maç sonuçlarının bulunduğu sayfayı çeker.
        url = f'https://www.skysports.com/real-madrid-results/{year_str}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Her bir maçın yer aldığı HTML bloklarını bulur.
        matches = soup.find_all('div', class_='fixres__item')

        # Her bir maçın verilerini işler.
        match_data = []
        for match in matches:
            # Takımların isimlerini bulur.
            teams = match.find_all('span', class_='swap-text__target')
            team_A = teams[0].text
            team_B = teams[1].text

            # Maç skorlarını bulur.
            scores = match.find('span', class_='matches__teamscores').text.strip()
            score_split = scores.split()
            score_A = score_split[0].strip()
            score_B = score_split[-1].strip()

            # Maç verilerini bir sözlükte toplar ve maç veri listesine ekler.
            match_data.append({
                'match_year': year_str,
                'team_A': team_A if 'Real Madrid' in team_A else team_B,
                'score_A': score_A if 'Real Madrid' in team_A else score_B,
                'team_B': team_B if 'Real Madrid' in team_A else team_A,
                'score_B': score_B if 'Real Madrid' in team_A else score_A,
            })

        # Oluşturulan maç veri listesi genel maç veri listesine eklenir.
        all_match_data.extend(match_data)

    # Toplanan tüm maç verileri bir DataFrame'e dönüştürülür.
    df = pd.DataFrame(all_match_data)
    
    # Gerekli sütunlar seçilerek bir CSV dosyasına kaydedilir.
    df[['match_year', 'team_A', 'team_B', 'score_A', 'score_B']].to_csv('./processed_matches.csv', index=False)
    print('processed_matches.csv dosyası oluşturuldu.')

# Fonksiyon çağrılır.
scrape_and_preprocess_matches()


processed_matches.csv dosyası oluşturuldu.


In [78]:
import pandas as pd

# Dosyayı oku
file_path = "./processed_matches.csv" 
df = pd.read_csv(file_path)

# 'result' sütununu ekleyerek veriyi güncelleme
df['result'] = df.apply(lambda row: 1 if row['score_A'] > row['score_B'] else
                                    0 if row['score_A'] < row['score_B'] else
                                    2 if row['score_A'] == row['score_B'] else
                                    None, axis=1)

# Güncellenmiş veriyi aynı dosyaya yazma
df.to_csv(file_path, index=False)
